In [2]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# Load preprocessed data
df = pd.read_parquet("real_estate_embeddings.parquet")
embeddings = np.stack(df["embedding"].values)

# Build FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# Load embedding model
embedder = SentenceTransformer("all-MiniLM-L6-v2")

def search_properties(query, top_k=5):
    query_embed = embedder.encode(query)
    distances, indices = index.search(np.array([query_embed]), top_k)
    results = df.iloc[indices[0]]
    return results[[
        'price', 'num_bed_rooms', 'num_bath_rooms', 'area', 
        'locality', 'city', 'property_url', 'image_url'
    ]]


In [3]:

# Example usage
results = search_properties("3-bed apartment under in Dhaka")
print(results)

            price  num_bed_rooms  num_bath_rooms    area  locality   city  \
1942    5900000.0            3.0             3.0  1100.0  Banasree  Dhaka   
14820     30000.0            3.0             3.0  1300.0     Badda  Dhaka   
23111  15000000.0            3.0             3.0  2000.0     Ramna  Dhaka   
1768   13200000.0            3.0             3.0  1600.0  Banasree  Dhaka   
23565  24952000.0            4.0             3.0  3119.0     Ramna  Dhaka   

                                            property_url  \
1942   https://www.bproperty.com/en/property/details-...   
14820  https://www.bproperty.com/en/property/details-...   
23111      https://pbazaar.com//en/apartment-for-sell-68   
1768   https://www.bproperty.com/en/property/details-...   
23565     https://pbazaar.com//en/nice-flat-for-sale-247   

                                               image_url  
1942   https://images-cdn.bproperty.com/thumbnails/15...  
14820  https://images-cdn.bproperty.com/thumbnails/71...  